### Explicación del Código

El código en `creador_historias.ipynb` procesa un archivo CSV con información sobre personas desaparecidas. Realiza las siguientes tareas:

1. **Creación de carpetas**:
   - Crea la carpeta `input_folder` si no existe y solicita al usuario que coloque los archivos CSV en ella.
   - Crea la carpeta `output` para guardar los resultados procesados.

2. **Selección de archivo CSV**:
   - Si no hay archivos CSV en `input_folder`, muestra un mensaje al usuario.
   - Si hay un solo archivo, lo selecciona automáticamente.
   - Si hay múltiples archivos, solicita al usuario que elija uno.

3. **Filtrado de casos**:
   - Lee el archivo CSV y selecciona registros donde `condicion_localizacion` es "NO APLICA".

4. **Anonimización**:
   - Envía las descripciones de desaparición al servicio DeepSeek para reescribirlas como historias breves y anonimizadas.

5. **Actualización de datos**:
   - Agrega las descripciones anonimizadas en una nueva columna `descripcion_anonimizada` y elimina la columna original `descripcion_desaparicion`.

6. **Exportación**:
   - Guarda los casos procesados en un nuevo archivo CSV en la carpeta `output` con un nombre que incluye la fecha y hora.

#### Carpetas Relevantes:
- **Entrada**: `input_folder` (debes colocar los archivos CSV aquí).
- **Salida**: `output` (los archivos procesados se guardan aquí).

#### Dependencias:
- Librerías estándar: `os`, `csv`, `random`, `logging`, `datetime`.
- Librería externa: `openai` (para interactuar con el servicio DeepSeek).

In [ ]:
# Instalar dependencias necesarias
!pip install openai

In [2]:
import os
import csv
import random
import logging
from datetime import datetime
from openai import OpenAI

# Configuración del cliente de DeepSeek
client = OpenAI(api_key="sk-aa403ef8a7b447d39df2a61893551166", base_url="https://api.deepseek.com")  # Reemplaza con tu API key

# Configuración de logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def anonymize_case(case):
    """Send the case description, name, and age to DeepSeek for rewriting."""
    try:
        logging.debug(f"Sending case to DeepSeek: {case}")
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "Eres un asistente útil que reescribe descripciones de desaparición como historias "
                        "breves pero que transmiten lo que pasó en español. Inspiras un poco de melancolía "
                        "pero no drama. Narras una desaparición."
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"Por favor, reescribe esta descripción como una historia MUY breve, repito muy breve."
                        f"No agregues título, menciona la fecha y la hora en tu respuesta. La historia debe narrar "
                        f"la desaparición de {case['nombre_completo']}, quien tenía {case['edad_momento_desaparicion']} "
                        f"años al momento de su desaparición. Remueve cualquier numero de casa, placa de auto o dato "
                        f"sensible que veas. Además, anonimiza los nombres de cualquier otra persona mencionada en la "
                        f"descripción, pero deja intacto el nombre {case['nombre_completo']} y presentalo en Sentencecase. "
                        f"Descripción: {case['descripcion_desaparicion']}."
                    )
                },
            ]
        )
        rewritten_story = response.choices[0].message.content
        logging.debug(f"Received response from DeepSeek: {rewritten_story}")
        return rewritten_story
    except Exception as e:
        logging.error(f"Error communicating with DeepSeek: {e}")
        return f"Error comunicándose con DeepSeek: {e}"

def process_csv(input_file, output_folder, num_cases):
    """Process the input CSV, filter cases, anonymize them, and save to a new CSV."""
    try:
        # Leer el archivo CSV
        with open(input_file, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            cases = []

            # Filtrar casos que cumplen con las condiciones
            for row in reader:
                if (row['condicion_localizacion'].strip().upper() == "NO APLICA"):
                    cases.append(row)

        logging.info(f"Total cases matching criteria: {len(cases)}")

        # Seleccionar casos aleatorios
        if len(cases) < num_cases:
            logging.warning(f"Only {len(cases)} cases available, selecting all.")
            selected_cases = cases
        else:
            selected_cases = random.sample(cases, num_cases)

        # Procesar cada caso y agregar la columna 'descripcion_anonimizada'
        for case in selected_cases:
            case['descripcion_anonimizada'] = anonymize_case(case)
            # Eliminar la columna 'descripcion_desaparicion'
            if 'descripcion_desaparicion' in case:
                del case['descripcion_desaparicion']

        # Crear la carpeta de salida si no existe
        os.makedirs(output_folder, exist_ok=True)

        # Generar el nombre del archivo con fecha y hora
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        output_file = os.path.join(output_folder, f"processed_cases_{timestamp}.csv")

        # Escribir los casos procesados en un nuevo archivo CSV
        with open(output_file, 'w', encoding='utf-8', newline='') as csvfile:
            fieldnames = list(selected_cases[0].keys())
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(selected_cases)

        logging.info(f"Archivo procesado y guardado en: {output_file}")
    except Exception as e:
        logging.error(f"Error processing CSV: {e}")

def get_input_file(input_folder):
    """Ensure the input folder exists and prompt the user to select a file if multiple CSVs are found."""
    os.makedirs(input_folder, exist_ok=True)
    csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

    if not csv_files:
        print(f"No CSV files found in {input_folder}.")
        print("Please place your CSV files in this folder and run the script again.")
        return None
    elif len(csv_files) == 1:
        return os.path.join(input_folder, csv_files[0])
    else:
        print("Multiple CSV files found. Please select one:")
        for i, file in enumerate(csv_files, 1):
            print(f"{i}. {file}")
        while True:
            try:
                choice = int(input("Enter the number of the file to use: "))
                if 1 <= choice <= len(csv_files):
                    return os.path.join(input_folder, csv_files[choice - 1])
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number.")

if __name__ == "__main__":
    input_folder = 'input_folder'
    output_folder = 'output'

    input_csv = get_input_file(input_folder)
    if input_csv:
        try:
            num_cases = int(input("Enter the number of cases to process: "))
            process_csv(input_csv, output_folder, num_cases)
        except ValueError:
            logging.error("Invalid input. Please enter a valid number.")

2025-04-13 16:13:13,543 - INFO - Total cases matching criteria: 5
2025-04-13 16:13:13,544 - DEBUG - Sending case to DeepSeek: {'id_cedula_busqueda': '93fbc3a7-07ab-59db-836c-87167e6d8487', 'autorizacion_informacion_publica': 'SI', 'condicion_localizacion': 'NO APLICA', 'nombre_completo': 'MARICELA SALAZAR MARTINEZ', 'edad_momento_desaparicion': '35', 'sexo': 'MUJER', 'genero': 'MUJER', 'complexion': 'DELGADA', 'estatura': '1.62', 'tez': 'BLANCO', 'cabello': 'LACIO, CASTAÑO, AL HOMBRO', 'ojos_color': 'CAFE OSCUROS', 'municipio': 'TLAJOMULCO DE ZUÑIGA', 'estado': 'JALISCO', 'fecha_desaparicion': '2023-12-05', 'estatus_persona_desaparecida': 'PERSONA DESAPARECIDA', 'descripcion_desaparicion': 'REFIERE LA REPORTANTE QUE EL DÍA 5 DICIEMBRE 2023 APROXIMADAMENTE 06:00 PM, SU HERMANA MANTUVO EL ÚLTIMO CONTACTO CON SU PROGENITORA LLAMADA ESTHER MARTÍNEZ VALDEZ (3314497134) POR LLAMADA TELEFÓNICA DEL NÚMERO  3326333862 (SOLICITABA CELULARES PRESTADOS PARA PODERSE COMUNICAR), MENCIONANDO QUE DEBE